# Service → Install Conversion Funnel Analysis
**Director of Analytics | Garage Door Rollup**

### Business Question
> *For customers who first came in for a service call, how many go on to book a Replacement/Install — and how long does it take?*

### Methodology
We use **Kaplan-Meier survival analysis** rather than a simple conversion rate.  
A plain rate ignores *time* and *censoring* — customers who haven't converted **yet** but still might.  
KM correctly accounts for this, giving us unbiased conversion probability estimates at any time horizon.

### Design Decisions
| Parameter | Choice |
|---|---|
| Conversion scope | Same brand only |
| Customer ID | `customer_id` as-is |
| Study end / censoring cutoff | 2026-01-01 |
| Funnel entry | Customer's **first-ever** service invoice at a given brand |
| Conversion event | Any subsequent `Replacement / Install` invoice at the **same brand** |

---
## Section 1 — Load & Clean Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'white',
    'axes.facecolor': '#f8f8f8',
    'axes.grid': True,
    'grid.alpha': 0.4,
    'font.size': 11
})

STUDY_END = pd.Timestamp('2026-01-01')
DATA_PATH = '/mnt/user-data/uploads/sample.xlsx'

print('Loading data...')
raw = pd.read_excel(DATA_PATH)
print(f'Raw shape: {raw.shape}')

In [ ]:
# ── 1.1  Basic cleaning ───────────────────────────────────────────────────────
df = raw.copy()

# Standardise column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Drop rows with null dates or customer IDs
missing_before = len(df)
df = df.dropna(subset=['invoice_date', 'customer_id'])
print(f'Dropped {missing_before - len(df):,} rows with null date or customer_id')

# Enforce date type
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

# Exclude invoices after study end (future-dated)
future = df[df['invoice_date'] > STUDY_END]
df = df[df['invoice_date'] <= STUDY_END].copy()
print(f'Excluded {len(future):,} invoices after study end ({STUDY_END.date()})')

# Flag negative / zero amount invoices for awareness (keep them — they're real events)
print(f'\nNegative amount invoices : {(df["total_amount"] < 0).sum():,}')
print(f'Zero amount invoices     : {(df["total_amount"] == 0).sum():,}')

# Keep only Completed statuses for revenue integrity; keep all for event detection
print(f'\nOrder status breakdown:')
print(df['order_status'].value_counts().to_string())

In [ ]:
# ── 1.2  Label invoice types we care about ───────────────────────────────────
SERVICE_TYPE     = 'Service'
INSTALL_TYPE     = 'Replacement / Install'

df['is_service'] = df['market_type'] == SERVICE_TYPE
df['is_install'] = df['market_type'] == INSTALL_TYPE

# Exclude cancelled orders from event detection
df_valid = df[df['order_status'] != 'Canceled'].copy()

print(f'Valid (non-cancelled) invoices : {len(df_valid):,}')
print(f'  → Service invoices           : {df_valid["is_service"].sum():,}')
print(f'  → Install invoices           : {df_valid["is_install"].sum():,}')
print(f'\nBrands in dataset:')
print(df_valid['brand'].value_counts().to_string())

---
## Section 2 — Cohort Construction

For each `(customer_id, brand)` pair we:
1. Find their **first service call date** → funnel entry
2. Exclude customers who had an install *before* their first service call (not in our funnel)
3. Find their **first install date** *after* the service call → conversion event
4. Customers without a subsequent install are **censored** at `min(last_invoice_date, STUDY_END)`

In [ ]:
# ── 2.1  Find first service date per (customer, brand) ───────────────────────
service_invoices = df_valid[df_valid['is_service']].copy()

first_service = (
    service_invoices
    .groupby(['customer_id', 'brand'])['invoice_date']
    .min()
    .reset_index()
    .rename(columns={'invoice_date': 'first_service_date'})
)

print(f'Unique (customer, brand) pairs with ≥1 service call: {len(first_service):,}')

In [ ]:
# ── 2.2  Find first install BEFORE first service (exclusion criterion) ────────
install_invoices = df_valid[df_valid['is_install']].copy()

first_install = (
    install_invoices
    .groupby(['customer_id', 'brand'])['invoice_date']
    .min()
    .reset_index()
    .rename(columns={'invoice_date': 'first_install_date'})
)

# Merge service cohort with install dates
cohort = first_service.merge(first_install, on=['customer_id', 'brand'], how='left')

# Exclude customers whose first install predates or equals their first service
prior_install = cohort[
    cohort['first_install_date'].notna() &
    (cohort['first_install_date'] <= cohort['first_service_date'])
]
print(f'Excluded {len(prior_install):,} (customer, brand) pairs — install predated service')

cohort = cohort[
    cohort['first_install_date'].isna() |
    (cohort['first_install_date'] > cohort['first_service_date'])
].copy()

print(f'Funnel cohort size: {len(cohort):,} (customer, brand) pairs')

In [ ]:
# ── 2.3  Build survival columns ───────────────────────────────────────────────

# Last observed date per (customer, brand) for censoring
last_obs = (
    df_valid
    .groupby(['customer_id', 'brand'])['invoice_date']
    .max()
    .reset_index()
    .rename(columns={'invoice_date': 'last_obs_date'})
)
cohort = cohort.merge(last_obs, on=['customer_id', 'brand'], how='left')

# Event flag: did they convert?
cohort['converted'] = cohort['first_install_date'].notna().astype(int)

# Time-to-event in days
# Converters   → days from first service to first install
# Non-converters → days from first service to min(last_obs, study_end)  [censored]
cohort['event_date'] = np.where(
    cohort['converted'] == 1,
    cohort['first_install_date'],
    cohort[['last_obs_date']].assign(end=STUDY_END).min(axis=1)
)
cohort['event_date'] = pd.to_datetime(cohort['event_date'])
cohort['days_to_event'] = (cohort['event_date'] - cohort['first_service_date']).dt.days

# Remove any zero or negative durations (same-day installs — edge case)
same_day = (cohort['days_to_event'] <= 0).sum()
cohort = cohort[cohort['days_to_event'] > 0].copy()
print(f'Removed {same_day} same-day or negative duration records')

print(f'\nFinal cohort: {len(cohort):,} records')
print(f'Converters  : {cohort["converted"].sum():,}  ({cohort["converted"].mean()*100:.1f}% raw rate)')
print(f'Censored    : {(cohort["converted"]==0).sum():,}')
print(f'\nTime-to-convert (converters only, days):')
print(cohort[cohort['converted']==1]['days_to_event'].describe().round(1).to_string())

In [ ]:
# ── 2.4  Enrich cohort with brand/market metadata ────────────────────────────

# Attach overall_market from the customer's first service invoice
market_lookup = (
    service_invoices
    .sort_values('invoice_date')
    .groupby(['customer_id', 'brand'])[['overall_market', 'location']]
    .first()
    .reset_index()
)
cohort = cohort.merge(market_lookup, on=['customer_id', 'brand'], how='left')

# Attach install revenue for converters
install_revenue = (
    install_invoices
    .sort_values('invoice_date')
    .groupby(['customer_id', 'brand'])['total_amount']
    .first()  # first install amount only
    .reset_index()
    .rename(columns={'total_amount': 'install_revenue'})
)
cohort = cohort.merge(install_revenue, on=['customer_id', 'brand'], how='left')

print('Cohort columns:', cohort.columns.tolist())
print(f'\nCohort by brand:')
print(
    cohort.groupby('brand')
    .agg(customers=('customer_id','count'), converters=('converted','sum'))
    .assign(raw_rate=lambda x: (x['converters']/x['customers']*100).round(1))
    .sort_values('customers', ascending=False)
    .to_string()
)

---
## Section 3 — Kaplan-Meier Survival Analysis

KM estimates the **probability of NOT converting** by time T.  
We flip this to get **cumulative conversion probability** = 1 − S(t).

$$\hat{S}(t) = \prod_{t_i \leq t} \left(1 - \frac{d_i}{n_i}\right)$$

Where $d_i$ = conversions at time $t_i$, $n_i$ = customers still at risk.

In [ ]:
# ── 3.1  Kaplan-Meier implementation ─────────────────────────────────────────

def kaplan_meier(durations, events, label='KM'):
    """
    Compute Kaplan-Meier survival curve with 95% Greenwood confidence intervals.
    
    Parameters
    ----------
    durations : array-like, time-to-event or censoring (days)
    events    : array-like, 1 = event (converted), 0 = censored
    
    Returns
    -------
    DataFrame with columns: time, n_risk, n_events, survival, ci_lower, ci_upper
    """
    d = pd.DataFrame({'t': durations, 'e': events}).sort_values('t').reset_index(drop=True)
    n_total = len(d)
    
    timeline = []
    S = 1.0
    greenwood_sum = 0.0  # Σ d_i / (n_i * (n_i - d_i))  for Greenwood variance
    
    # Process each unique event time
    event_times = sorted(d[d['e'] == 1]['t'].unique())
    
    for t in event_times:
        n_risk   = (d['t'] >= t).sum()          # at risk just before t
        n_events = ((d['t'] == t) & (d['e'] == 1)).sum()  # events at t
        
        if n_risk == 0:
            continue
        
        S = S * (1 - n_events / n_risk)
        
        # Greenwood's formula for variance of log(S)
        denom = n_risk * (n_risk - n_events)
        if denom > 0:
            greenwood_sum += n_events / denom
        
        # 95% CI via log(-log(S)) transformation (more stable near boundaries)
        if S > 0 and S < 1 and greenwood_sum > 0:
            se_log_log_S = np.sqrt(greenwood_sum / (np.log(S) ** 2))
            log_log_S    = np.log(-np.log(S))
            ci_lower = np.exp(-np.exp(log_log_S + 1.96 * se_log_log_S))
            ci_upper = np.exp(-np.exp(log_log_S - 1.96 * se_log_log_S))
        else:
            ci_lower = ci_upper = S
        
        timeline.append({
            'time'    : t,
            'n_risk'  : n_risk,
            'n_events': n_events,
            'survival': S,
            'ci_lower': np.clip(ci_lower, 0, 1),
            'ci_upper': np.clip(ci_upper, 0, 1)
        })
    
    result = pd.DataFrame(timeline)
    # Add t=0 anchor
    result = pd.concat([
        pd.DataFrame([{'time':0,'n_risk':n_total,'n_events':0,'survival':1.0,'ci_lower':1.0,'ci_upper':1.0}]),
        result
    ], ignore_index=True)
    
    result['conversion_prob'] = 1 - result['survival']
    result['conv_ci_lower']   = 1 - result['ci_upper']  # flip for conversion
    result['conv_ci_upper']   = 1 - result['ci_lower']
    result.attrs['label'] = label
    return result


def prob_at(km_df, days):
    """Interpolate conversion probability at a given number of days."""
    row = km_df[km_df['time'] <= days].tail(1)
    if row.empty:
        return 0.0
    return float(row['conversion_prob'].values[0])


def median_time(km_df):
    """Return median time-to-convert (days where conversion prob ≥ 0.50), or None."""
    crossed = km_df[km_df['conversion_prob'] >= 0.50]
    if crossed.empty:
        return None
    return int(crossed.iloc[0]['time'])


print('KM functions defined ✓')

In [ ]:
# ── 3.2  Overall KM curve ─────────────────────────────────────────────────────
km_overall = kaplan_meier(cohort['days_to_event'], cohort['converted'], label='All Brands')

# Key milestones
milestones = [180, 365, 548, 730]  # 6mo, 12mo, 18mo, 24mo
print('=== Overall Conversion Probabilities ===')
for d in milestones:
    p = prob_at(km_overall, d)
    print(f'  {d//30:2d} months ({d} days): {p*100:.1f}%')

med = median_time(km_overall)
if med:
    print(f'\nMedian time to convert: {med} days ({med/30:.1f} months)')
else:
    print(f'\nMedian not reached (conversion probability never exceeds 50%)')
    p_max = km_overall['conversion_prob'].max()
    print(f'Max conversion probability in study window: {p_max*100:.1f}%')

In [ ]:
# ── 3.3  KM by Brand ─────────────────────────────────────────────────────────
brands = cohort['brand'].value_counts()
# Only include brands with enough data for meaningful curves (≥100 customers)
brands_to_plot = brands[brands >= 100].index.tolist()

km_by_brand = {}
for brand in brands_to_plot:
    sub = cohort[cohort['brand'] == brand]
    km_by_brand[brand] = kaplan_meier(sub['days_to_event'], sub['converted'], label=brand)

print(f'Brands with ≥100 funnel customers: {brands_to_plot}')

# ── 3.4  KM by Market Type (Residential vs Commercial) ───────────────────────
# Roll up to simpler buckets
market_map = {
    'Residential': 'Residential',
    'Commercial':  'Commercial',
    'Mixed':       'Mixed',
}
cohort['market_bucket'] = cohort['overall_market'].map(market_map).fillna('Other')

km_by_market = {}
for mkt, sub in cohort.groupby('market_bucket'):
    if len(sub) >= 50:
        km_by_market[mkt] = kaplan_meier(sub['days_to_event'], sub['converted'], label=mkt)

print(f'Market buckets: {list(km_by_market.keys())}')

---
## Section 4 — Visualisations

In [ ]:
# ── 4.1  Overall KM conversion curve ─────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 6))

# Convert days → months for readability
t_months = km_overall['time'] / 30.44
p        = km_overall['conversion_prob'] * 100
lo       = km_overall['conv_ci_lower'] * 100
hi       = km_overall['conv_ci_upper'] * 100

ax.step(t_months, p, where='post', color='#1f77b4', lw=2.5, label='Conversion probability')
ax.fill_between(t_months, lo, hi, step='post', alpha=0.2, color='#1f77b4', label='95% CI')

# Milestone markers
colors_m = ['#d62728', '#ff7f0e', '#2ca02c', '#9467bd']
labels_m = ['6 mo', '12 mo', '18 mo', '24 mo']
for i, (days, lbl, c) in enumerate(zip(milestones, labels_m, colors_m)):
    p_m = prob_at(km_overall, days)
    ax.axvline(days/30.44, color=c, linestyle='--', alpha=0.6, lw=1.2)
    ax.annotate(f'{lbl}\n{p_m*100:.1f}%',
                xy=(days/30.44, p_m*100),
                xytext=(days/30.44 + 0.8, p_m*100 + 1.5),
                fontsize=9, color=c,
                arrowprops=dict(arrowstyle='->', color=c, lw=1))

ax.set_xlabel('Months since first service call', fontsize=12)
ax.set_ylabel('Cumulative conversion probability (%)', fontsize=12)
ax.set_title('Service → Install Conversion Curve (All Brands)\nKaplan-Meier Estimate with 95% Confidence Interval',
             fontsize=13, fontweight='bold')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_xlim(0)
ax.set_ylim(0)
ax.legend(fontsize=10)
plt.tight_layout()
plt.savefig('/home/claude/fig_overall_km.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved fig_overall_km.png')

In [ ]:
# ── 4.2  KM curves by Brand ───────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(13, 7))

palette = plt.cm.tab10.colors
for i, (brand, km) in enumerate(km_by_brand.items()):
    t_m = km['time'] / 30.44
    p   = km['conversion_prob'] * 100
    ax.step(t_m, p, where='post', lw=2, color=palette[i], label=brand)

# 12-month reference line
ax.axvline(12, color='grey', linestyle=':', lw=1.5, label='12-month mark')

ax.set_xlabel('Months since first service call', fontsize=12)
ax.set_ylabel('Cumulative conversion probability (%)', fontsize=12)
ax.set_title('Service → Install Conversion by Brand\nKaplan-Meier Estimates', fontsize=13, fontweight='bold')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_xlim(0)
ax.set_ylim(0)
ax.legend(fontsize=9, loc='upper left')
plt.tight_layout()
plt.savefig('/home/claude/fig_brand_km.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── 4.3  KM curves by Market Type ────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 6))

mkt_colors = {'Residential': '#1f77b4', 'Commercial': '#d62728', 'Mixed': '#2ca02c', 'Other': '#9467bd'}
for mkt, km in km_by_market.items():
    t_m = km['time'] / 30.44
    p   = km['conversion_prob'] * 100
    lo  = km['conv_ci_lower'] * 100
    hi  = km['conv_ci_upper'] * 100
    c   = mkt_colors.get(mkt, '#7f7f7f')
    ax.step(t_m, p, where='post', lw=2.5, color=c, label=mkt)
    ax.fill_between(t_m, lo, hi, step='post', alpha=0.12, color=c)

ax.axvline(12, color='grey', linestyle=':', lw=1.5)
ax.set_xlabel('Months since first service call', fontsize=12)
ax.set_ylabel('Cumulative conversion probability (%)', fontsize=12)
ax.set_title('Service → Install Conversion by Market Type\nKaplan-Meier Estimates with 95% CI', fontsize=13, fontweight='bold')
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
ax.set_xlim(0)
ax.set_ylim(0)
ax.legend(fontsize=10)
plt.tight_layout()
plt.savefig('/home/claude/fig_market_km.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Section 5 — Revenue Impact Analysis

Conversion rates are only half the story — we also want to know **what a conversion is worth**.

In [ ]:
# ── 5.1  Install revenue by brand & market ───────────────────────────────────
converters = cohort[cohort['converted'] == 1].copy()
converters = converters[converters['install_revenue'] > 0]  # exclude zero/negative

print('=== Install Revenue (First Install, Converters Only) ===')
print(f'\nOverall median install value : ${converters["install_revenue"].median():,.0f}')
print(f'Overall mean install value   : ${converters["install_revenue"].mean():,.0f}')

rev_by_brand = (
    converters.groupby('brand')['install_revenue']
    .agg(['median', 'mean', 'count'])
    .rename(columns={'median':'median_$', 'mean':'mean_$', 'count':'n_converters'})
    .sort_values('median_$', ascending=False)
    .round(0)
)
print('\n=== By Brand ===')
print(rev_by_brand.to_string())

In [ ]:
# ── 5.2  Expected Revenue Per Service Customer (ERPSC) ────────────────────────
# ERPSC = conversion_probability_at_24mo × median_install_value
# This is the expected revenue attributable to acquiring one service customer.

print('=== Expected Revenue Per Service Customer @ 24 months ===')
print(f'{" ":<20} {"Conv% (24mo)":>14} {"Median Install":>16} {"ERPSC":>10} {"Cohort N":>10}')
print('-' * 75)

erpsc_rows = []
for brand in sorted(km_by_brand.keys()):
    km   = km_by_brand[brand]
    conv = prob_at(km, 730)  # 24 months
    sub  = converters[converters['brand'] == brand]
    med_rev = sub['install_revenue'].median() if len(sub) > 5 else np.nan
    erpsc   = conv * med_rev if not np.isnan(med_rev) else np.nan
    n       = len(cohort[cohort['brand'] == brand])
    
    erpsc_rows.append({'brand': brand, 'conv_24mo': conv, 'median_install': med_rev,
                       'erpsc': erpsc, 'cohort_n': n})
    
    med_str  = f'${med_rev:,.0f}' if not np.isnan(med_rev) else 'N/A'
    erp_str  = f'${erpsc:,.0f}' if not np.isnan(erpsc) else 'N/A'
    print(f'{brand:<20} {conv*100:>13.1f}% {med_str:>16} {erp_str:>10} {n:>10,}')

erpsc_df = pd.DataFrame(erpsc_rows)

In [ ]:
# ── 5.3  ERPSC bar chart ──────────────────────────────────────────────────────
plot_df = erpsc_df.dropna(subset=['erpsc']).sort_values('erpsc', ascending=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Conversion rate @ 24 months
ax = axes[0]
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.9, len(plot_df)))
bars = ax.barh(plot_df['brand'], plot_df['conv_24mo'] * 100, color=colors)
ax.bar_label(bars, fmt='%.1f%%', padding=3, fontsize=9)
ax.set_xlabel('Conversion probability (%)')
ax.set_title('24-Month Conversion Rate by Brand', fontweight='bold')
ax.xaxis.set_major_formatter(mtick.PercentFormatter())

# Right: ERPSC
ax = axes[1]
bars = ax.barh(plot_df['brand'], plot_df['erpsc'], color=colors)
ax.bar_label(bars, fmt='$%.0f', padding=3, fontsize=9)
ax.set_xlabel('Expected Revenue Per Service Customer ($)')
ax.set_title('ERPSC @ 24 Months by Brand', fontweight='bold')
ax.xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'${x:,.0f}'))

plt.suptitle('Brand Scorecard: Conversion Rate & Revenue Impact', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('/home/claude/fig_erpsc.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Section 6 — "Most Likely to Convert Soon" Customer List

Operational output: identify unconverted service customers who are entering the **highest-velocity conversion window** based on the KM curve — prime targets for proactive outreach.

In [ ]:
# ── 6.1  Find the peak conversion velocity window ────────────────────────────
# Velocity = change in conversion probability per 30-day window
km_v = km_overall.copy()
km_v['time_months'] = km_v['time'] / 30.44

# Bin into monthly buckets
km_v['month_bin'] = km_v['time_months'].astype(int)
monthly = (
    km_v.groupby('month_bin')['conversion_prob']
    .max()
    .diff()
    .fillna(0)
)
peak_month = monthly.idxmax()
print(f'Peak conversion velocity: month {peak_month} (days {peak_month*30}–{(peak_month+1)*30})')
print(f'Incremental conversions that month: {monthly[peak_month]*100:.2f} percentage points')

# Define outreach window: customers whose time-since-service falls within ±3 months of peak
OUTREACH_LOW_DAYS  = max(0, (peak_month - 3) * 30)
OUTREACH_HIGH_DAYS = (peak_month + 3) * 30
print(f'\nOutreach window: {OUTREACH_LOW_DAYS}–{OUTREACH_HIGH_DAYS} days since first service')

In [ ]:
# ── 6.2  Build the outreach list ──────────────────────────────────────────────

# As-of date for age calculation: use study end
non_converters = cohort[cohort['converted'] == 0].copy()
non_converters['days_since_service'] = (STUDY_END - non_converters['first_service_date']).dt.days

# Filter to the outreach window
outreach = non_converters[
    (non_converters['days_since_service'] >= OUTREACH_LOW_DAYS) &
    (non_converters['days_since_service'] <= OUTREACH_HIGH_DAYS)
].copy()

# Score each customer: survival probability remaining (how much conversion potential is left?)
# Higher residual conversion probability = higher priority
outreach['remaining_conv_potential'] = outreach['days_since_service'].apply(
    lambda d: (
        km_overall[km_overall['time'] <= OUTREACH_HIGH_DAYS + 365]['conversion_prob'].max() -
        prob_at(km_overall, d)
    )
)

# Sort by brand + remaining potential
outreach = outreach.sort_values(['brand', 'remaining_conv_potential'], ascending=[True, False])

outreach_export = outreach[[
    'customer_id', 'brand', 'location', 'overall_market',
    'first_service_date', 'days_since_service', 'remaining_conv_potential'
]].copy()
outreach_export['remaining_conv_potential'] = (outreach_export['remaining_conv_potential'] * 100).round(1)
outreach_export.columns = [
    'Customer ID', 'Brand', 'Location', 'Market',
    'First Service Date', 'Days Since Service', 'Remaining Conv Potential (%)'
]

print(f'Customers in outreach window: {len(outreach_export):,}')
print(f'\nTop 20 by remaining conversion potential:')
print(outreach_export.head(20).to_string(index=False))

---
## Section 7 — Summary Scorecard

In [ ]:
# ── 7.1  Full brand scorecard ─────────────────────────────────────────────────
scorecard_rows = []

for brand in sorted(km_by_brand.keys()):
    km  = km_by_brand[brand]
    sub = cohort[cohort['brand'] == brand]
    conv_sub = converters[converters['brand'] == brand]
    
    row = {
        'Brand'               : brand,
        'Funnel Entrants'     : len(sub),
        'Converters'          : sub['converted'].sum(),
        'Raw Conv %'          : f"{sub['converted'].mean()*100:.1f}%",
        'KM 6-mo %'           : f"{prob_at(km, 180)*100:.1f}%",
        'KM 12-mo %'          : f"{prob_at(km, 365)*100:.1f}%",
        'KM 24-mo %'          : f"{prob_at(km, 730)*100:.1f}%",
        'Median Install $'    : f"${conv_sub['install_revenue'].median():,.0f}" if len(conv_sub) > 5 else 'N/A',
        'ERPSC @ 24mo'        : erpsc_df[erpsc_df['brand']==brand]['erpsc'].values[0]
    }
    row['ERPSC @ 24mo'] = f"${row['ERPSC @ 24mo']:,.0f}" if not np.isnan(row['ERPSC @ 24mo']) else 'N/A'
    scorecard_rows.append(row)

scorecard = pd.DataFrame(scorecard_rows)
print('=== BRAND SCORECARD: Service → Install Funnel ===')
print(scorecard.to_string(index=False))

In [ ]:
# ── 7.2  Export outreach list to CSV ─────────────────────────────────────────
outreach_export.to_csv('/mnt/user-data/outputs/service_to_install_outreach_list.csv', index=False)
scorecard.to_csv('/mnt/user-data/outputs/brand_scorecard.csv', index=False)
print('Exported:')
print('  → /mnt/user-data/outputs/service_to_install_outreach_list.csv')
print('  → /mnt/user-data/outputs/brand_scorecard.csv')

In [ ]:
# ── 7.3  Final summary dashboard ──────────────────────────────────────────────
fig = plt.figure(figsize=(16, 10))
gs  = GridSpec(2, 2, figure=fig, hspace=0.4, wspace=0.35)

# Panel A: Overall KM
ax_a = fig.add_subplot(gs[0, :])
t_m = km_overall['time'] / 30.44
p   = km_overall['conversion_prob'] * 100
lo  = km_overall['conv_ci_lower'] * 100
hi  = km_overall['conv_ci_upper'] * 100
ax_a.step(t_m, p, where='post', color='#1f77b4', lw=2.5)
ax_a.fill_between(t_m, lo, hi, step='post', alpha=0.2, color='#1f77b4')
for days, lbl, c in zip(milestones, labels_m, colors_m):
    p_m = prob_at(km_overall, days)
    ax_a.axvline(days/30.44, color=c, ls='--', alpha=0.5, lw=1)
    ax_a.text(days/30.44 + 0.3, p_m*100 + 0.5, f'{lbl}: {p_m*100:.1f}%', color=c, fontsize=8)
ax_a.set(xlabel='Months since first service', ylabel='Conversion %',
         title='Service → Install Conversion (All Brands) — Kaplan-Meier')
ax_a.yaxis.set_major_formatter(mtick.PercentFormatter())
ax_a.set_xlim(0); ax_a.set_ylim(0)

# Panel B: Conversion rate by brand @ 12 months
ax_b = fig.add_subplot(gs[1, 0])
bdf = erpsc_df.dropna(subset=['erpsc']).copy()
bdf['km_12mo'] = bdf['brand'].apply(lambda b: prob_at(km_by_brand[b], 365) * 100)
bdf_s = bdf.sort_values('km_12mo')
c_bars = plt.cm.RdYlGn(np.linspace(0.2, 0.9, len(bdf_s)))
bars = ax_b.barh(bdf_s['brand'], bdf_s['km_12mo'], color=c_bars)
ax_b.bar_label(bars, fmt='%.1f%%', padding=2, fontsize=8)
ax_b.set(xlabel='12-month conversion rate', title='Conversion Rate @ 12 Mo by Brand')
ax_b.xaxis.set_major_formatter(mtick.PercentFormatter())

# Panel C: ERPSC by brand
ax_c = fig.add_subplot(gs[1, 1])
bdf_s2 = bdf.sort_values('erpsc')
bars = ax_c.barh(bdf_s2['brand'], bdf_s2['erpsc'], color=c_bars)
ax_c.bar_label(bars, fmt='$%.0f', padding=2, fontsize=8)
ax_c.set(xlabel='Expected revenue per service customer', title='ERPSC @ 24 Months by Brand')
ax_c.xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'${x:,.0f}'))

fig.suptitle('Service → Install Funnel: Executive Summary Dashboard',
             fontsize=14, fontweight='bold', y=1.01)
plt.savefig('/mnt/user-data/outputs/summary_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()
print('Dashboard saved.')

---
## Key Takeaways

Run the notebook on your full dataset and fill in these cells with findings.

| Metric | Value |
|---|---|
| Total funnel cohort | *(from Section 2)* |
| KM-adjusted 12-month conversion rate | *(from Section 3)* |
| KM-adjusted 24-month conversion rate | *(from Section 3)* |
| Median time to convert | *(from Section 3)* |
| Median install value | *(from Section 5)* |
| Best converting brand | *(from Section 7)* |
| Customers in outreach window | *(from Section 6)* |

### Next Steps
- **Cox Proportional Hazards model** → add covariates (market type, season, brand) to find drivers of faster conversion
- **Log-rank test** → statistically confirm whether brand differences are significant
- **RFM segmentation** → layer customer value scoring on top of the outreach list